In [2]:
print("hello world")

hello world


# 初期設定

In [3]:
# 必要なライブラリのインポート
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader


In [4]:
import os
import sys

# プロジェクトのルートディレクトリを設定
root_dir = os.path.abspath(os.path.join(os.getcwd(), '..'))
sys.path.append(os.path.join(root_dir, 'src'))

# データの準備

In [5]:
transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5,), (0.5,))])

train_dataset = datasets.MNIST(root='./data', train=True, download=True, transform=transform)
train_loader = DataLoader(train_dataset, batch_size=64, shuffle=True)

test_dataset = datasets.MNIST(root='./data', train=False, download=True, transform=transform)
test_loader = DataLoader(test_dataset, batch_size=64, shuffle=False)

# CNNモデルの定義

In [6]:
class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        
        # 畳み込み層
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=32, kernel_size=3, padding=1)  # 入力: 1チャネル (グレースケール画像)
        self.conv2 = nn.Conv2d(in_channels=32, out_channels=64, kernel_size=3, padding=1)
        
        # プーリング層
        self.pool = nn.MaxPool2d(kernel_size=2, stride=2)
        
        # 全結合層
        self.fc1 = nn.Linear(64 * 7 * 7, 128)  # 画像のサイズが28x28 -> 14x14 -> 7x7 after pooling
        self.fc2 = nn.Linear(128, 10)  # 10クラスの分類
        
    def forward(self, x):
        x = self.pool(torch.relu(self.conv1(x)))  # 畳み込み -> ReLU -> プーリング
        x = self.pool(torch.relu(self.conv2(x)))  # 畳み込み -> ReLU -> プーリング
        x = x.view(-1, 64 * 7 * 7)  # フラット化
        x = torch.relu(self.fc1(x))  # 全結合層
        x = self.fc2(x)  # 出力層
        return x


# モデル、損失関数、最適化手法の設定

In [7]:
model = CNN()
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

# 訓練ループ

In [8]:
num_epochs = 5

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    for images, labels in train_loader:
        optimizer.zero_grad()
        
        outputs = model(images)
        loss = criterion(outputs, labels)
        loss.backward()
        
        optimizer.step()
        
        running_loss += loss.item()

    print(f"Epoch [{epoch+1}/{num_epochs}], Loss: {running_loss/len(train_loader):.4f}")

Epoch [1/5], Loss: 0.1567
Epoch [2/5], Loss: 0.0456
Epoch [3/5], Loss: 0.0309
Epoch [4/5], Loss: 0.0239
Epoch [5/5], Loss: 0.0163


# モデル評価

In [9]:
model.eval()  # 評価モードに変更
correct = 0
total = 0
with torch.no_grad():  # 勾配計算を行わない
    for images, labels in test_loader:
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += labels.size(0)
        correct += (predicted == labels).sum().item()

accuracy = 100 * correct / total
print(f"Test Accuracy: {accuracy:.2f}%")


Test Accuracy: 98.30%
